## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 115.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 257.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 249.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 203.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.3.1 which is incompatible.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.3.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 

In [2]:
# For installing the libraries & downloading models from HF Hub
# !pip install huggingface_hub==0.28.1 pandas==2.2.2 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.3.26 langchain-community==0.3.26 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==2.1.0 -q
!pip install huggingface_hub==0.28.1 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
blosc2 3.5.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is inco

In [3]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import numpy as np
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [4]:
# Loading the LLM from Hugging Face by giving the model name & basename
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename) # Defining the model path to be downloaded from Hugging Face

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
llm = Llama(model_path=model_path, n_threads=2, n_batch=512, n_gpu_layers=43, n_ctx=4096) # Defining the hyper parameters for the model llm along with the model path downloaded from Hugging face

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [6]:
def response(query,max_tokens=1000,temperature=0,top_p=0.95,top_k=50): # Creating a function to define the model parameters and generate a response

    # Generating an input from the model
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,        # maximum tokens to be generated
      temperature=temperature,      # randomeness of the generated response
      top_p=top_p,                  # diversity of the generated response
      top_k=top_k,                  # maximum number of most likely new tokens to consider
      repeat_penalty=1.2,           # penalty for repeating tokens in generated response
      echo=True,                    # Whether to return the input prompt to model
      stop=["INST"]                 # list of tokens used to dynamically stop response generation
    )

    return model_output['choices'][0]['text']     # Return the response text as output of the function

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [7]:
Query1 = "What is the protocol for managing sepsis in a critical care unit?"     # Defining the Query1 so that it can be used in the future
Response1 = response(Query1,max_tokens=1000,temperature=0,top_p=0.95,top_k=50)   # Defining the Response so that it can be used for comparision in the future
print(Response1)                                                                 # Printing the Response

What is the protocol for managing sepsis in a critical care unit?

Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care setting:

1. Early Recognition: Identify patients who may be at risk of developing sepsis based on clinical signs such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis. Use validated scoring systems like the Sequential Organ Failure Assessment (SOFA) score to help identify patients with sepsis.
2. Source Control: Identify and treat the source of infection as soon as possible. This may involve surgical intervention or other interventions such as drainage, debridement, or antibiotics.
3. Fluid Resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion and prevent hypotension. The goal is to achieve a mean

**Observation**: The response generated gives us protocols for managing sepsis as asked. It also provides information on what Sepsis is and its symptoms for Early Recognition etc. The response seems to be very comprehensive.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [8]:
Query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?" # Defining the Query2 so that it can be used in the future
Response2 = response(Query2,max_tokens=1000,temperature=0,top_p=0.95,top_k=50)   # Defining the Response2 so that it can be used for comparision in the future
print(Response2)                                                                 # Printing the Response

Llama.generate: prefix-match hit


What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Appendicitis is a medical condition characterized by inflammation of the appendix, a small pouch-like structure that extends from the large intestine. The symptoms of appendicitis typically include:

1. Abdominal pain: This is usually located in the lower right side of the abdomen and may start as a mild discomfort but can quickly progress to become severe and constant.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain or nausea.
3. Nausea and vomiting: Vomiting is common in appendicitis, especially if the inflammation presses on the stomach.
4. Fever: A fever above 101°F (38.3°C) may be present due to the body's response to infection.
5. Diarrhea or constipation: Depending on the location and pressure of inflammation, people with appendicitis might experience diarrhea or constipation.
6. Rebound te

**Observation**: The response generated answers all the questions asked and in a distinctive manner for each question. The response also seems to explain all answers adequately. In addtion it has answered what Appendicitis is, which was not asked as a question based on context.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [9]:
Query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?" # Defining the Query3 so that it can be used in the future
Response3 = response(Query3,max_tokens=1000,temperature=0,top_p=0.95,top_k=50)   # Defining the Response3 so that it can be used for comparision in the future
print(Response3)                                                                 # Printing the Response

Llama.generate: prefix-match hit


What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects approximately 1-2% of the population. It can occur at any age, but it's more prevalent in children and young adults. The exact cause of alopecia areata is not fully understood, but it's believed to be related to an abnormal immune response where the body attacks its own hair follicles.

There are several treatments for addressing sudden patchy hair loss:

1. Corticosteroids: These medications can help reduce inflammation and suppress the immune system, allowing the hair follicles to regrow. They can be applied topically or taken orally in pill form. Topical corticosteroid creams or ointments are often used for small patches of alopecia areata, while oral medications may be prescribed for larger areas 

**Observation**: The response generated distinctly answers both questions adequately with a clear bifurcation and has also identified the ailment to be Alopecia areata which makes it even a better response.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [10]:
Query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"  # Defining the Query4 so that it can be used in the future
Response4 = response(Query4,max_tokens=1000,temperature=0,top_p=0.95,top_k=50)  # Defining the Response4 so that it can be used for comparision in the future
print(Response4)                                                                # Printing the Response

Llama.generate: prefix-match hit


What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

A person with a brain injury may require various treatments depending on the severity and location of the injury. Here are some common treatments:

1. Emergency care: In case of a traumatic brain injury (TBI), it is essential to seek immediate medical attention. The primary goal in emergency care is to prevent further damage, stabilize vital signs, and manage any life-threatening conditions such as bleeding or swelling in the brain.
2. Medications: Depending on the symptoms of a brain injury, medications may be prescribed to help manage various aspects of recovery. For instance, anti-inflammatory drugs can reduce inflammation, while anticonvulsants are used to prevent seizures. Pain relievers and sedatives might also be necessary for managing discomfort or agitation.
3. Rehabilitation: Brain injury rehabilitation is a cruci

**Observation**: The response generated has covered several bases from immediate surgery options, medication options to post surgery recommendations. The overall response gives a very good and detailed answer to the query given.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [11]:
Query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"  # Defining the Query5 so that it can be used in the future
Response5 = response(Query5,max_tokens=1000,temperature=0,top_p=0.95,top_k=50)  # Defining the Response5 so that it can be used for comparision in the future
print(Response5)                                                                # Printing the Response

Llama.generate: prefix-match hit


What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

First and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:

1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and swelling. Avoid moving them unless absolutely necessary to prevent worsening the fracture or causing additional injuries.
2. Assess for other injuries: Check for any signs of other injuries, such as head trauma, chest pain, or difficulty breathing. If you suspect that there are other injuries, prioritize their treatment and seek medical help immediately.
3. Provide first aid: Apply a sterile dressing to the fracture site if it's open or bleeding. Use a splint or sling to immobilize the leg as much as possible while still all

**Observation**: The response generated clearly describes the necessary precautions and the recovery options which are 2 different questions and 2 separate answers are received.

**Summary**: Overall, looking at the responses generated, although they are effective in responding to the queries, they seem to be exhaustive and generic. The responses need to be briefer and more technical as these are generated for health care professionals and not general audience.

## Question Answering using LLM with Prompt Engineering

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [12]:
system_prompt = "Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list" # Defining the prompt to be provided to the system to get a better response

In [13]:
Eng_query1 = system_prompt + "\n" + Query1 + "\n\n"                                       # New query incudes Prompt Engineering
Eng_response1 = response(Eng_query1,max_tokens=1000,temperature=0.5,top_p=0.80,top_k=50)  # Temprature increased to 0.5 from 0.0 and top_p decreased from 0.95 to 0.80
print(Eng_response1)                                                                      # Printing the Response

Llama.generate: prefix-match hit


Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What is the protocol for managing sepsis in a critical care unit?

1. Early recognition: Recognize signs of infection, organ dysfunction, and inflammation as early as possible using validated scoring systems like Sequential Organ Failure Assessment (SOFA) or Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA).
2. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion, aiming for a mean arterial pressure of 65 mmHg and central venous oxygen saturation above 70%.
3. Antibiotic administration: Start broad-spectrum antibiotics within one hour of recognition based on suspected infection site and microbiology data.
4. Source control: Identify and address the source of infection promptly, such as surgical debridement or drainage procedures.
5. Vasopressor support: U

In [14]:
print(Response1)                                                                          # Printing the earlier response without prompt Engineering or LLM parameter tuning for comparision

What is the protocol for managing sepsis in a critical care unit?

Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care setting:

1. Early Recognition: Identify patients who may be at risk of developing sepsis based on clinical signs such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis. Use validated scoring systems like the Sequential Organ Failure Assessment (SOFA) score to help identify patients with sepsis.
2. Source Control: Identify and treat the source of infection as soon as possible. This may involve surgical intervention or other interventions such as drainage, debridement, or antibiotics.
3. Fluid Resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion and prevent hypotension. The goal is to achieve a mean

**Observation**:

*  Due to Prompt Engineering the response is more concise - lines have become shorter than the original response and few more technical details have been added.
*  When Temprature was increased from 0.0 to 0.5 to increase randomness and top_p decreased from 0.95 to 0.80 which is still higher we don't see much of a change in the response as almost the same pointers have been generated again.  

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
Eng_query2 = system_prompt + "\n" + Query2 + "\n\n"                                       # New query incudes Prompt Engineering
Eng_response2 = response(Eng_query2,max_tokens=1000,temperature=0.0,top_p=0.80,top_k=20)  # top_k decreased to 20 from 50 and top_p decreased from 0.95 to 0.80
print(Eng_response2)                                                                      # Printing the Response

Llama.generate: prefix-match hit


Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

1. Appendicitis is characterized by:
   - Periumbilical or right lower quadrant abdominal pain that worsens with movement or coughing
   - Loss of appetite and nausea/vomiting
   - Fever (often low-grade)
   - Constipation or diarrhea
2. Appendicitis cannot be cured via medicine alone as the appendix will eventually rupture if left untreated, leading to peritonitis which is a medical emergency.
3. The standard surgical procedure for treating appendicitis is an appendectomy:
   - Open appendectomy: A small incision is made in the lower right abdomen and the appendix is removed.
   - Laparoscopic appendectomy: Several small incisions are made, and a laparoscope (a thin tube w

In [16]:
print(Response2)                                                                          # Printing the earlier response without prompt Engineering or LLM parameter tuning for comparision

What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

Appendicitis is a medical condition characterized by inflammation of the appendix, a small pouch-like structure that extends from the large intestine. The symptoms of appendicitis typically include:

1. Abdominal pain: This is usually located in the lower right side of the abdomen and may start as a mild discomfort but can quickly progress to become severe and constant.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal pain or nausea.
3. Nausea and vomiting: Vomiting is common in appendicitis, especially if the inflammation presses on the stomach.
4. Fever: A fever above 101°F (38.3°C) may be present due to the body's response to infection.
5. Diarrhea or constipation: Depending on the location and pressure of inflammation, people with appendicitis might experience diarrhea or constipation.
6. Rebound te

**Observation**:

*  Due to Prompt Engineering the response is more concise and technical which helathcare professionals understand rather than long chains to text.
*  By decreasing top_k to 20 from 50 and decreasing top_p from 0.95 to 0.80 we have ensured responses to be less random which helps here as we need more accurate information than creative information.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [17]:
Eng_query3 = system_prompt + "\n" + Query3 + "\n\n"                                       # New query incudes Prompt Engineering
Eng_response3 = response(Eng_query3,max_tokens=1000,temperature=0.5,top_p=0.95,top_k=50)  # temprature increased from 0.0 to 0.5
print(Eng_response3)                                                                      # Printing the Response

Llama.generate: prefix-match hit


Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

1. Minoxidil: A topical medication approved by the FDA for treating male and female pattern hair loss. It can stimulate new hair growth in affected areas when applied twice daily to the bald spot.
2. Corticosteroids: Prescription medications used to reduce inflammation, suppress the immune system, and promote hair regrowth. They are effective in addressing alopecia areata, an autoimmune condition causing sudden patchy hair loss.
3. DHT blockers: Finasteride (Proscar, Propecia) or dutasteride (Avodart) can be prescribed to inhibit the production of dihydrotestosterone (DHT), a hormone that contributes to male pattern baldness and an

In [18]:
print(Response3)                                                                          # Printing the earlier response without prompt Engineering or LLM parameter tuning for comparision

What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects approximately 1-2% of the population. It can occur at any age, but it's more prevalent in children and young adults. The exact cause of alopecia areata is not fully understood, but it's believed to be related to an abnormal immune response where the body attacks its own hair follicles.

There are several treatments for addressing sudden patchy hair loss:

1. Corticosteroids: These medications can help reduce inflammation and suppress the immune system, allowing the hair follicles to regrow. They can be applied topically or taken orally in pill form. Topical corticosteroid creams or ointments are often used for small patches of alopecia areata, while oral medications may be prescribed for larger areas 

**Observation**:

*  Increasing Temprature to 0.5 from 0.0, we see that the randomness has increased with no clear bifurcation between responses for both questions asked which was present earlier. Also, although most pointers are mentioned in Response3, few new pointers like Minoxidil and Finasteride are mentioned in Eng_response3.
*  Prompt Engineering has managed to reduce the content slightly and made it more technical for healthcare professionals to understand.



### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [19]:
Eng_query4 = system_prompt + "\n" + Query4 + "\n\n"                                       # New query incudes Prompt Engineering
Eng_response4 = response(Eng_query4,max_tokens=256,temperature=0.0,top_p=0.80,top_k=50)   # max_tokens decreased from 1000 to 256, top_p decreased to 0.95 from 0.80
print(Eng_response4)                                                                      # Printing the Response

Llama.generate: prefix-match hit


Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

1. Rehabilitation therapy: This includes occupational therapy, speech-language therapy, and physical therapy to help the patient regain lost skills and improve overall functioning.
2. Medications: Depending on the specific symptoms, medications may be prescribed to manage seizures, control agitation or aggression, reduce muscle spasms, or alleviate pain.
3. Surgery: In some cases, surgery may be recommended to remove hematomas (blood clots) or repair damaged blood vessels in the brain.
4. Assistive devices and technology: Devices such as wheelchairs, walkers, communication aids, and assistive technology can help improve mobility, communication, and indepe

In [20]:
print(Response4)                                                                          # Printing the earlier response without prompt Engineering or LLM parameter tuning for comparision

What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

A person with a brain injury may require various treatments depending on the severity and location of the injury. Here are some common treatments:

1. Emergency care: In case of a traumatic brain injury (TBI), it is essential to seek immediate medical attention. The primary goal in emergency care is to prevent further damage, stabilize vital signs, and manage any life-threatening conditions such as bleeding or swelling in the brain.
2. Medications: Depending on the symptoms of a brain injury, medications may be prescribed to help manage various aspects of recovery. For instance, anti-inflammatory drugs can reduce inflammation, while anticonvulsants are used to prevent seizures. Pain relievers and sedatives might also be necessary for managing discomfort or agitation.
3. Rehabilitation: Brain injury rehabilitation is a cruci

**Observation**:

*   Prompt Engineering has removed some of the unnecessary information when looking from a physician / healthcare professional point of view and made the response crisper.
*   When max_tokens were decreased from 1000 to 256 the response remained incomplete which can be seen with the incomplete sentence formed for Vocational rehabilitation.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [21]:
Eng_query5 = system_prompt + "\n" + Query5 + "\n\n"                                       # New query incudes Prompt Engineering
Eng_response5 = response(Eng_query5,max_tokens=1000,temperature=0.2,top_p=0.80,top_k=30)  # temprature increased from 0.0 to 0.2, top_k decreased to 30 from 50 and top_p decreased from 0.95 to 0.80
print(Eng_response5)                                                                      # Printing the Response

Llama.generate: prefix-match hit


Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

1. Assess the severity of the injury: Determine if it's an open or closed fracture, and check for signs of nerve damage or impaired circulation.
2. Immobilize the affected limb using a splint or cast to prevent further movement and promote healing.
3. Apply ice packs intermittently to reduce swelling and pain.
4. Elevate the leg above heart level whenever possible to minimize swelling and improve blood flow.
5. Administer appropriate pain medication as prescribed by a healthcare professional.
6. Seek medical attention promptly for proper diagnosis, treatment, and follow-up care.
7. Monitor for signs of infection such as redness, warmth, or in

In [22]:
print(Response5)                                                                          # Printing the earlier response without prompt Engineering or LLM parameter tuning for comparision

What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

First and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:

1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and swelling. Avoid moving them unless absolutely necessary to prevent worsening the fracture or causing additional injuries.
2. Assess for other injuries: Check for any signs of other injuries, such as head trauma, chest pain, or difficulty breathing. If you suspect that there are other injuries, prioritize their treatment and seek medical help immediately.
3. Provide first aid: Apply a sterile dressing to the fracture site if it's open or bleeding. Use a splint or sling to immobilize the leg as much as possible while still all

**Observation**:

*  With a combination of increasing Temprature slightly to 0.2 from 0.0, top_k decreased to 30 from 50 and top_p decreased slightly to 0.80 the language has become more technical i.e., suitable for a healthcare professional removing points like Seek medical help and Provide first aid and adding things like Transportation, Pain Management and Nutrition & Hydration.
*  Prompt Engineering although removes the bifurcation between responses to the 2 questions, but it has made the text more nuanced in terms of medical usage and much shorter. It has also removed the redundant information like Emotional support and added prescriptive points like Avoid weight bearing activities.

## Data Preparation for RAG

### Loading the Data

In [23]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Loading the document
pdf_loader = PyMuPDFLoader("/content/drive/MyDrive/Python - Google Colab/5. Natural Language Processing NLP/Project - Medical Assistant/medical_diagnosis_manual.pdf")  # Defining the path and Retrieving the document using PyMuPDFLoader
merck_manual = pdf_loader.load()

Document has been loaded onto the system.

### Data Overview

#### Checking the first 5 pages

In [25]:
# Displaying the first 5 pages of the file loaded
for i in range(5):
    print("Page Number:", i+1, "\n")                                # To print the page number at the beginning
    print(merck_manual[i].page_content, "\n")                       # To print the content of the page

Page Number: 1 

shreekumarsahu11@gmail.com
Q4GFM2CUV7
for personal use by shreekumarsahu11@
shing the contents in part or full is liable 
 

Page Number: 2 

shreekumarsahu11@gmail.com
Q4GFM2CUV7
This file is meant for personal use by shreekumarsahu11@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
 

Page Number: 3 

Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .....................................................................................................................................................................

#### Checking the number of pages

In [26]:
len(merck_manual)                                                               # Checking the number of pages in the merck manual

4114

### Data Chunking

In [27]:
# Initializing a Text splitter which can split text to chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(           # We are using Tiktoken encoding for token based splitting
    encoding_name='cl100k_base',                                                # specifies the Tiktoken encoding used by OpenAI models
    chunk_size=512,                                                             # Each chunk will have 512 tokens maximum
    chunk_overlap=20,                                                           # Overlap of 20 tokens between consecutive chunks
    )

In [28]:
merck_manual_chunks = pdf_loader.load_and_split(text_splitter)               # Splitting the document into chunks

In [29]:
i = 100
merck_manual_chunks[i]                                                       # Printing the 100th chunk

Document(page_content='compatible with good health). Vegans may develop vitamin B12 deficiency unless they consume yeast\nextracts or Asian-style fermented foods. Their intake of Ca, iron, and zinc also tends to be low. A fruit-only\ndiet is not recommended because it is deficient in protein, Na, and many micronutrients.\nFad diets: Some fad diets result in vitamin, mineral, and protein deficiencies; cardiac, renal, and\nmetabolic disorders; and sometimes death. Very low calorie diets (< 400 kcal/day) cannot sustain health\nfor long.\nDrugs and nutritional supplements: Many drugs (eg, appetite suppressants, digoxin) decrease\nappetite; others impair nutrient absorption or metabolism. Some drugs (eg, stimulants) have catabolic\neffects. Certain drugs can impair absorption of many nutrients; eg, anticonvulsants can impair absorption\nof vitamins.\nAlcohol or drug dependency: Patients with alcohol or drug dependency may neglect their nutritional\nneeds. Absorption and metabolism of nutrie

In [30]:
merck_manual_chunks[i+1]                                                       # Printing the 101st chunk

Document(page_content="Diagnosis is based on results of medical and diet histories, physical examination, body composition\nanalysis (see p. 58), and selected laboratory tests.\nHistory: History should include questions about dietary intake (see\nFig. 2-1), recent changes in weight, and risk factors for undernutrition, including drug and alcohol use.\nUnintentional loss of ≥ 10% of usual body weight during a 3-mo period indicates a high probability of\nundernutrition. Social history should include questions about whether money is available for food and\nwhether the patient can shop and cook.\nReview of systems should focus on symptoms of nutritional deficiencies (see Table 2-1). For example,\nimpaired night vision may indicate vitamin A deficiency.\nPhysical examination: Physical examination should include measurement of height and weight,\ninspection of body fat distribution, and anthropometric measurements of lean body mass. Body mass\nindex (BMI = weight(kg)/height(m)2) adjusts weig

In [31]:
len(merck_manual_chunks)  # No of chunks created from the merck_manual document

8497

A total of 8497 chunks have been created from the 4114 page Merck Manual with each chunk having 512 tokens and 20 overlap tokens from previous chunk.

### Embedding

In [32]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large') # Defining the embedding model

In [33]:
embedding_1 = embedding_model.embed_query(merck_manual_chunks[0].page_content)    # Defining the embedding of the 1st chunk to be compared later
embedding_2 = embedding_model.embed_query(merck_manual_chunks[1].page_content)    # Defining the embedding of the 2nd chunk to be compared later

In [34]:
print("Dimension of the vector ",len(embedding_1))                                # Identifying the dimension of vector for 1st chunk

Dimension of the vector  1024


In [35]:
len(embedding_1)==len(embedding_2)                                                # Comparing the vector dimensions for 1st and 2nd chunk if they are same

True

Embedding for each chunk was done with a vector dimension of 1024 and we also verified the dimensions for the 1st 2 embeddings are same.

### Vector Database

In [36]:
folder_name = 'merck_manual_database'   # Assigning the name of the folder / directory as merck_manual_database

if not os.path.exists(folder_name):     # Checking if the folder / directory name already exists
    os.makedirs(folder_name)            # Creating the folder / directory merck_manual_database if it doesnt already exist

In [37]:
Vector_db = Chroma.from_documents(      # Creating a vector database
    documents=merck_manual_chunks,      # Defining the chunks created from Merck Manual
    embedding=embedding_model,          # Defining the embedding model to be used
    persist_directory=folder_name,      # Defining the directory where the vectors to be stored
    )

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [38]:
Vector_db = Chroma(persist_directory=folder_name, embedding_function=embedding_model)  # Loading Chroma vector database created with chunk embeddings from the specified directory with the given embedding model

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [39]:
Vector_db.embeddings                                # Viewing the embedding function used in the Chroma vector store

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [40]:
Vector_db.similarity_search("appendicitis",k=3)     # Finding out the top 3 similar chunks to the input - appendicitis

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(page_content="Appendicitis\nAppendicitis is acute inflammation of the vermiform appendix, typically resulting in abdominal\npain, anorexia, and abdominal tenderness. Diagnosis is clinical, often supplemented by CT or\nultrasound. Treatment is surgical removal.\nIn the US, acute appendicitis is the most common cause of acute abdominal pain requiring surgery. Over\n5% of the population develops appendicitis at some point. It most commonly occurs in the teens and 20s\nbut may occur at any age.\nOther conditions affecting the appendix include carcinoids, cancer, villous adenomas, and diverticula. The\nappendix may also be affected by Crohn's disease or ulcerative colitis with pancolitis.\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nChapter 11. Acute Abdomen & Surgical Gastroenterology\n163\nshreekumarsahu11@gmail.com\nQ4GFM2CUV7\nThis file is meant for personal use by shreekumarsahu11@gmail.com only.\nSharing or publishing the contents in part or full is liable for leg

A Vector Database called Vector_db was created to store the embeddings for all chunks created from the Merck Manual to be used later as part of the context to be shared with the LLM.

### Retriever

In [41]:
Retriever = Vector_db.as_retriever(             # Defining the retriever
    search_type="similarity",                   # Defining the search type as similarity
    search_kwargs={"k":3}                       # To Retrieve the top 3 similar chunks for a given query
)

In [42]:
Example_retrieve = Retriever.get_relevant_documents(query="What are the common symptoms for sepsis")  # Using the Retriever to answer a query
Example_retrieve                                                                                      # Printing the top 3 contextually similar retrieved documents / chunks

[Document(page_content='overwhelming, severe infections, profound neutropenia is often a poor prognostic sign. Characteristic\nmorphologic changes in the neutrophils of septic patients include Dohle bodies, toxic granulations, and\nvacuolization.\nAnemia can develop despite adequate tissue iron stores. If anemia is chronic, plasma iron and total iron-\nbinding capacity may be decreased. Serious infection, particularly with gram-negative organisms, may\ncause disseminated intravascular coagulation (DIC—see p. 976).\nOther organ systems: Pulmonary compliance may decrease, progressing to acute respiratory distress\nsyndrome (ARDS) and respiratory muscle failure.\nRenal manifestations range from minimal proteinuria to acute renal failure, which can result from shock\nand acute tubular necrosis, glomerulonephritis, or tubulointerstitial disease.\nHepatic dysfunction, including cholestatic jaundice (often a poor prognostic sign) or hepatocellular\ndysfunction, occurs with many infections, ev

Retriever function was defined to retireive k similar chunks to a query to be fed as external context to LLM.

### System and User Prompt Template

In [43]:
# Providing the System Message
qna_system_message = """
You are a helpful assistant trained to give answers to questions asked with repect to a context.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
User questions will begin with the token: ###Question.
The context contains references to specific portions of a document relevant to the Question.
Strictly answer only using the information provided in the ###Context.
If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".
"""

In [44]:
# Creating a template for Context and Query by user i.e. user_input
qna_user_message_template = """
###Context
{context}

###Question
{question}
"""

### Response Function

In [45]:
def generate_rag_response(user_input,k=3,max_tokens=1000,temperature=0,top_p=0.95,top_k=50):     # Defining a function to generate a RAG response
    global qna_system_message,qna_user_message_template                                         # Here qna_system_message and qna_user_message_template are defined as Global variables and not specific only to defined function
    # Retrieve relevant document chunks
    relevant_document_chunks = Retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

System Message and User message were defined so that they can be provided to the LLM as the external context so that more nuanced responses can be generated.

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [46]:
RAG_response1 = generate_rag_response(Query1)                                            # Generating the response to Query using the RAG model defined above
print(RAG_response1)                                                                     # Printing the response generated by RAG model

Llama.generate: prefix-match hit


Based on the context, the protocol for managing sepsis in a critical care unit includes:
1. Prompt empiric antibiotic therapy: Start immediately after suspecting sepsis.
2. Selection of antibiotics: Based on suspected source, clinical setting, knowledge or suspicion of causative organisms and sensitivity patterns common to that specific inpatient unit, and previous culture results.
3. Regimens for septic shock of unknown cause: Gentamicin or tobramycin 5.1 mg/kg IV once/day plus a 3rd-generation cephalosporin (cefotaxime 2 g q 6 to 8 h or ceftriaxone 2 g once/day) or ceftazidime 2 g IV q 8 h, or ceftazidime plus a fluoroquinolone (eg, ciprofloxacin).
4. Adding vancomycin if resistant staphylococci or enterococci are suspected.
5. Draining abscesses and excising necrotic tissues.
6. Normalizing blood glucose: A continuous IV insulin infusion is titrated to maintain glucose between 80 to 110 mg/dL (4.4 to 6.1 mmol/L).
7. Corticosteroid therapy: With replacement doses rather than pharmaco

In [47]:
print(Eng_response1)                                    # Printing the earlier Prompt Engineered response for Query1 for comparision

Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What is the protocol for managing sepsis in a critical care unit?

1. Early recognition: Recognize signs of infection, organ dysfunction, and inflammation as early as possible using validated scoring systems like Sequential Organ Failure Assessment (SOFA) or Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA).
2. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion, aiming for a mean arterial pressure of 65 mmHg and central venous oxygen saturation above 70%.
3. Antibiotic administration: Start broad-spectrum antibiotics within one hour of recognition based on suspected infection site and microbiology data.
4. Source control: Identify and address the source of infection promptly, such as surgical debridement or drainage procedures.
5. Vasopressor support: U

**Observation**: The RAG model response generated is based on the Merck Manual. While there are some similar pointers from the Prompt Engineered response, the new response seems to have more nuanced answer and detailed answer.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [48]:
RAG_response2 = generate_rag_response(Query2)                                            # Generating the response to Query using the RAG model defined above
print(RAG_response2)                                                                     # Printing the response generated by RAG model

Llama.generate: prefix-match hit


I don't know if appendicitis can be cured via medicine based on the provided context. The common symptoms for appendicitis include abdominal pain, anorexia, and abdominal tenderness (Merck Manual of Diagnosis & Therapy, 19th Edition). The treatment for appendicitis is surgical removal (open or laparoscopic appendectomy) due to the risk of mortality if left untreated (Merck Manual of Diagnosis & Therapy, 19th Edition).


In [49]:
print(Eng_response2)                                    # Printing the earlier Prompt Engineered response for Query2 for comparision

Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

1. Appendicitis is characterized by:
   - Periumbilical or right lower quadrant abdominal pain that worsens with movement or coughing
   - Loss of appetite and nausea/vomiting
   - Fever (often low-grade)
   - Constipation or diarrhea
2. Appendicitis cannot be cured via medicine alone as the appendix will eventually rupture if left untreated, leading to peritonitis which is a medical emergency.
3. The standard surgical procedure for treating appendicitis is an appendectomy:
   - Open appendectomy: A small incision is made in the lower right abdomen and the appendix is removed.
   - Laparoscopic appendectomy: Several small incisions are made, and a laparoscope (a thin tube w

**Observation**: It seems like the Merck Manual doesn't expressly mention whether Appendicitis can be cured via medicine hence the LLM cannot answer that. Regarding the symptoms and the surgical procedure the new response is able to answer in a concise manner.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [50]:
RAG_response3 = generate_rag_response(Query3)                                            # Generating the response to Query using the RAG model defined above
print(RAG_response3)                                                                     # Printing the response generated by RAG model

Llama.generate: prefix-match hit


Based on the context provided, the condition being described is alopecia areata. The effective treatments for this condition include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). The cause of alopecia areata is believed to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers.


In [51]:
print(Eng_response3)                                    # Printing the earlier Prompt Engineered response for Query3 for comparision

Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

1. Minoxidil: A topical medication approved by the FDA for treating male and female pattern hair loss. It can stimulate new hair growth in affected areas when applied twice daily to the bald spot.
2. Corticosteroids: Prescription medications used to reduce inflammation, suppress the immune system, and promote hair regrowth. They are effective in addressing alopecia areata, an autoimmune condition causing sudden patchy hair loss.
3. DHT blockers: Finasteride (Proscar, Propecia) or dutasteride (Avodart) can be prescribed to inhibit the production of dihydrotestosterone (DHT), a hormone that contributes to male pattern baldness and an

**Observation**: The RAG model is able to identify the condition as alopecia areata and prescribe effective treatments as well as possible causes in a concise manner which includes most of what the Prompt Engineered LLM suggested.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [52]:
RAG_response4 = generate_rag_response(Query4)                                            # Generating the response to Query using the RAG model defined above
print(RAG_response4)                                                                     # Printing the response generated by RAG model

Llama.generate: prefix-match hit


Based on the context provided, the recommended treatments for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function include:
1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.
2. Surgery if necessary to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.
3. Maintaining adequate brain perfusion and oxygenation in the first few days after the injury and preventing complications of altered sensorium.
4. Rehabilitation for many patients subsequently.
5. Supportive care including preventing systemic complications due to immobilization, providing good nutrition, and preventing pressure ulcers.
6. Joining support groups for assistance to families of brain-injured patients.
7. A prolonged period of rehabilitation, particularly in cognitive and emotional areas, is often requir

In [53]:
print(Eng_response4)                                    # # Printing the earlier Prompt Engineered response for Query4 for comparision

Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

1. Rehabilitation therapy: This includes occupational therapy, speech-language therapy, and physical therapy to help the patient regain lost skills and improve overall functioning.
2. Medications: Depending on the specific symptoms, medications may be prescribed to manage seizures, control agitation or aggression, reduce muscle spasms, or alleviate pain.
3. Surgery: In some cases, surgery may be recommended to remove hematomas (blood clots) or repair damaged blood vessels in the brain.
4. Assistive devices and technology: Devices such as wheelchairs, walkers, communication aids, and assistive technology can help improve mobility, communication, and indepe

**Observation**: When it comes to treatment methodology, the RAG system is able to provide more technical and to the point response based on the context provided than Prompt Engineered response.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [54]:
RAG_response5 = generate_rag_response(Query5)                                            # Generating the response to Query using the RAG model defined above
print(RAG_response5)                                                                     # Printing the response generated by RAG model

Llama.generate: prefix-match hit


Based on the context provided, here's the answer:

The necessary precautions for a person with a fractured leg include immobilizing the injured limb using either a cast or a splint. The joints proximal and distal to the injury should also be immobilized. If severe swelling is likely, the cast (and all padding) may be cut open from end to end medially and laterally (bivalved). Patients with casts should keep them dry, never put any object inside, inspect the edges and skin around the cast daily, apply lotion to any red or sore areas, pad any rough edges, and seek medical care at once if an odor emanates from within the cast or a fever develops.

The treatment steps for a person with a fractured leg include analgesics for pain relief, immobilization using a cast or splint, and sometimes surgery. Supportive measures and local wound care are also necessary. In the field, frostbitten extremities should be rewarmed rapidly by immersing them in water that is tolerably warm to the touch (40 to

In [55]:
print(Eng_response5)                                    # Printing the earlier Prompt Engineered response for Query5 for comparision

Answer the question in a more concise manner and the responses can be more technical which healthcare professionals understand. The response needs to be in a numbered list
What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

1. Assess the severity of the injury: Determine if it's an open or closed fracture, and check for signs of nerve damage or impaired circulation.
2. Immobilize the affected limb using a splint or cast to prevent further movement and promote healing.
3. Apply ice packs intermittently to reduce swelling and pain.
4. Elevate the leg above heart level whenever possible to minimize swelling and improve blood flow.
5. Administer appropriate pain medication as prescribed by a healthcare professional.
6. Seek medical attention promptly for proper diagnosis, treatment, and follow-up care.
7. Monitor for signs of infection such as redness, warmth, or in

**Observation**: The response from RAG model is more specific and comprehensive. The Prompt Engineered model comparatively seems to be slightly generic than RAG model response. However it is more structured and concise.

### Fine-tuning

In [56]:
# Finetuning RAG response for Query 1
Finetuning_RAG1 = generate_rag_response(Query1,k=3,max_tokens=1000,temperature=0.5,top_p=0.95,top_k=50)  # Temprature changed to 0.5 from 0.0.
print(Finetuning_RAG1)

Llama.generate: prefix-match hit


The context provides information on the management of sepsis in a critical care unit, including the use of antibiotics and the need to drain abscesses and excise necrotic tissues. The recommended antibiotic regimen includes gentamicin or tobramycin, a 3rd-generation cephalosporin, and vancomycin if resistant staphylococci or enterococci are suspected. It is important to note that when culture and sensitivity results are available, the antibiotic regimen should be changed accordingly. Antibiotics are continued for at least 5 days after shock resolves and evidence of infection subsides. Abscesses must be drained, and necrotic tissues must be surgically excised to prevent further deterioration of the patient's condition despite antibiotic therapy. The context also mentions that normalization of blood glucose improves outcome in critically ill patients and that corticosteroid therapy seems beneficial. However, there is no explicit mention of a specific protocol for managing sepsis in a cri

In [57]:
print(RAG_response1)                                    # Printing the earlier RAG response without Fine-tuning for comparision

Based on the context, the protocol for managing sepsis in a critical care unit includes:
1. Prompt empiric antibiotic therapy: Start immediately after suspecting sepsis.
2. Selection of antibiotics: Based on suspected source, clinical setting, knowledge or suspicion of causative organisms and sensitivity patterns common to that specific inpatient unit, and previous culture results.
3. Regimens for septic shock of unknown cause: Gentamicin or tobramycin 5.1 mg/kg IV once/day plus a 3rd-generation cephalosporin (cefotaxime 2 g q 6 to 8 h or ceftriaxone 2 g once/day) or ceftazidime 2 g IV q 8 h, or ceftazidime plus a fluoroquinolone (eg, ciprofloxacin).
4. Adding vancomycin if resistant staphylococci or enterococci are suspected.
5. Draining abscesses and excising necrotic tissues.
6. Normalizing blood glucose: A continuous IV insulin infusion is titrated to maintain glucose between 80 to 110 mg/dL (4.4 to 6.1 mmol/L).
7. Corticosteroid therapy: With replacement doses rather than pharmaco

**Observation**: By introducing a change in the temperature to 0.5 we see the response has become more arbitrary with respect to response before fine-tuning and the model output says there is no explicit mention of a specific protocol for managing sepsis in a critical care unit. It shows by reducing predictabiity in response and increasing randomness we are not getting a proper response. This means temperature value should be low.

In [58]:
# Finetuning RAG response for Query 2
Finetuning_RAG2 = generate_rag_response(Query2,k=5,max_tokens=1000,temperature=0.5,top_p=0.95,top_k=50)  # Temprature changed to 0.5 from 0.0 and value of k (retrieving top k similar documents) is changed from 3 to 5.
print(Finetuning_RAG2)

Llama.generate: prefix-match hit


I don't know if appendicitis can be cured via medicine based on the context provided. The common symptoms for appendicitis include abdominal pain, anorexia, and abdominal tenderness (Merck Manual of Diagnosis & Therapy, 19th Edition). Treatment for appendicitis is surgical removal (open or laparoscopic appendectomy) due to the risk of mortality increasing with treatment delay.


In [59]:
print(RAG_response2)                                    # Printing the earlier RAG response for Query2 without Fine-tuning for comparision

I don't know if appendicitis can be cured via medicine based on the provided context. The common symptoms for appendicitis include abdominal pain, anorexia, and abdominal tenderness (Merck Manual of Diagnosis & Therapy, 19th Edition). The treatment for appendicitis is surgical removal (open or laparoscopic appendectomy) due to the risk of mortality if left untreated (Merck Manual of Diagnosis & Therapy, 19th Edition).


**Observation**: Even by broadening the randomness by increasing temprature from 0.0 to 0.5 and increasing value of k from 3 to 5 to take into account more chunks of context, the model is not able to answer if appendicitis can be cured via medicine.

The response after fine-tuning is same. This shows there may be limited information about apendicitis in the Merck Manual.

In [60]:
# Finetuning RAG response for Query 3
Finetuning_RAG3 = generate_rag_response(Query3,k=3,max_tokens=1000,temperature=0.0,top_p=0.80,top_k=30)  # top_p changed to 0.8 from 0.95 and value of top_k is changed from 50 to 30.
print(Finetuning_RAG3)

Llama.generate: prefix-match hit


Based on the context provided, the condition being described is alopecia areata. The effective treatments for this condition include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). The cause of alopecia areata is believed to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers.


In [61]:
print(RAG_response3)                                    # Printing the earlier RAG response for Query3 without Fine-tuning for comparision

Based on the context provided, the condition being described is alopecia areata. The effective treatments for this condition include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA). The cause of alopecia areata is believed to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers.


**Observation**: By changing the top_p from 0.95 to 0.8 and value of top_k from 50 to 30, we do not see much of change in the response with respect to response before fine-tuning. It shows by reducing diversity of the output we are not able to change much which means there is limited information on the query in the chunks passed as context.

In [62]:
# Finetuning RAG response for Query 4
Finetuning_RAG4 = generate_rag_response(Query4,k=3,max_tokens=1000,temperature=0.2,top_p=0.95,top_k=20)  # top_k is changed from 50 to 20 and temprature is increased to 0.2.
print(Finetuning_RAG4)

Llama.generate: prefix-match hit


Based on the context provided, the recommended treatments for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function include:
1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.
2. Surgery if necessary to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.
3. Maintaining adequate brain perfusion and oxygenation in the first few days after the injury and preventing complications of altered sensorium.
4. Rehabilitation for many patients subsequently.
5. Supportive care which includes preventing systemic complications due to immobilization, providing good nutrition, and preventing pressure ulcers.
6. Joining support groups for assistance to families of brain-injured patients.
7. A prolonged period of rehabilitation, particularly in cognitive and emotional areas, is often r

In [63]:
print(RAG_response4)                                    # Printing the earlier RAG response for Query4 without Fine-tuning for comparision

Based on the context provided, the recommended treatments for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function include:
1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.
2. Surgery if necessary to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.
3. Maintaining adequate brain perfusion and oxygenation in the first few days after the injury and preventing complications of altered sensorium.
4. Rehabilitation for many patients subsequently.
5. Supportive care including preventing systemic complications due to immobilization, providing good nutrition, and preventing pressure ulcers.
6. Joining support groups for assistance to families of brain-injured patients.
7. A prolonged period of rehabilitation, particularly in cognitive and emotional areas, is often requir

**Observation**: By ensuring top_k is changed from 50 to 20 to reduce diversity and temprature is increased to 0.2 to introduce slight randomness we see that the new response is more concise, consists most elements from the RAG response before fine-tuning. Hence, reducing the top_k value helped the response.

In [64]:
# Finetuning RAG response for Query 5
Finetuning_RAG5 = generate_rag_response(Query5,k=5,max_tokens=1000,temperature=0.0,top_p=0.60,top_k=20)  # top_p changed to 0.60 from 0.95; top_k changed to 20 from 50; value of k is changed to 5 from 3
print(Finetuning_RAG5)

Llama.generate: prefix-match hit


Based on the context provided, here's the answer:

The necessary precautions for a person with a fractured leg include immobilizing the injured limb using either a cast or a splint. The joints proximal and distal to the injury should also be immobilized. If severe swelling is likely, the cast (and all padding) may be cut open from end to end medially and laterally (bivalved). Patients with casts should keep them dry, never put any object inside, inspect the edges and skin around the cast daily, apply lotion to any red or sore areas, pad any rough edges, and seek medical care at once if an odor emanates from within the cast or a fever develops.

The treatment steps for a person with a fractured leg include analgesics for pain relief, immobilization using a cast or splint, and sometimes surgery. Supportive measures and local wound care are also necessary. In the field, frostbitten extremities should be rewarmed rapidly by immersing them in water that is tolerably warm to the touch (40 to

In [65]:
print(RAG_response5)                                    # Printing the earlier RAG response for Query5 without Fine-tuning for comparision

Based on the context provided, here's the answer:

The necessary precautions for a person with a fractured leg include immobilizing the injured limb using either a cast or a splint. The joints proximal and distal to the injury should also be immobilized. If severe swelling is likely, the cast (and all padding) may be cut open from end to end medially and laterally (bivalved). Patients with casts should keep them dry, never put any object inside, inspect the edges and skin around the cast daily, apply lotion to any red or sore areas, pad any rough edges, and seek medical care at once if an odor emanates from within the cast or a fever develops.

The treatment steps for a person with a fractured leg include analgesics for pain relief, immobilization using a cast or splint, and sometimes surgery. Supportive measures and local wound care are also necessary. In the field, frostbitten extremities should be rewarmed rapidly by immersing them in water that is tolerably warm to the touch (40 to

**Observation**: In this case we have changed the top_k to 20 and restricted the token selection to only the top 60% to reduce diversity. Also value of k is increased from 3 to 5 so that more context can be taken. Changing these hyper parameters we see that the fine-tuned response gives the same output as the RAG response. Hence, there may not be sufficient context in the manual regarding the query.

## Output Evaluation

In [66]:
# Providing the input to model for the Groundedness rater system i.e., if answer has been derived from context.
groundedness_rater_system_message  = """
You will be presented with a ###Question, ###Context used by the AI system and AI generated ###Answer.
Your task is to evaluate if the ###Answer has been derived from the ###Context.
Rate 1 - if The ###Answer has not been derived from ###Context at all
Rate 2 - if The ###Answer has been derived from the ###Context only to a limited extent
Rate 3 - if The ###Answer has been derived from ###Context to a good extent
Rate 4 - if The ###Answer has been derived from ###Context mostly
Rate 5 - if The ###Answer has been derived from ###Context completely
Please ensure to give a single number as output among 1, 2, 3, 4, 5 along with a brief explanation for the output given.
"""

In [67]:
# Providing the input to model for the Relevance rater system i.e., if answer is relevant to the question.
relevance_rater_system_message = """
You will be presented with a ###Question, ###Context used by the AI system and AI generated ###Answer.
Your task is to evaluate the extent to which the ###Answer is relevant to the ###Question.
Rate 1 - if The ###Answer is not relevant to the ###Question at all
Rate 2 - if The ###Answer is missing key aspects and is relevant to the ###Question only to a limited extent
Rate 3 - if The ###Answer is covering some key aspects and is relevant to the ###Question to a good extent
Rate 4 - if The ###Answer is covering most key aspects and is relevant to the ###Question mostly
Rate 5 - if The ###Answer is covering all aspects and is fully relevant to the ###Question completely
Please ensure to give a single number as output among 1, 2, 3, 4, 5 along with a brief explanation for the output given.
"""

In [68]:
# Creating a template for providing Context and Query by user i.e. user_input
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [69]:
# Defining a function to evaluate the responses provided by the RAG model
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = Retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    # Response creation using LLM and defining LLM hyperparameters
    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Groundedness Response
    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )
    # Relevance Response
    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )
    # Output of the function
    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

In [70]:
# Output evaluation for Query 1: What is the protocol for managing sepsis in a critical care unit?
ground1,rel1 = generate_ground_relevance_response(Query1,max_tokens=256)            # Calling the function defined to evaluate the outputs
print(ground1,"\n\n")                                                               # Printing the Groundedness output
print(rel1)                                                                         # Printing the Relevance output

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Output: 5
Explanation: The answer has been derived from the context completely as it covers all the essential points mentioned in the context regarding the protocol for managing sepsis in a critical care unit. 


 Output: 5
Explanation: The AI-generated answer covers all aspects of the question regarding the protocol for managing sepsis in a critical care unit, including giving parenteral antibiotics after taking specimens, starting prompt empiric therapy, changing the antibiotic regimen based on culture and sensitivity results, various antibiotic regimens, adding Vancomycin or anaerobic drug if necessary, continuing antibiotics for a certain period, draining abscesses and excising necrotic tissues, and normalizing blood glucose and using corticosteroid therapy.


**Observation**: The groundedness rating is 5 and the relevance rating is 5. This means the RAG model is able to effectively use the context to generate the response and the response generated answers the question in its entirety as well.

In [71]:
# Output evaluation for Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
ground2,rel2 = generate_ground_relevance_response(Query2,max_tokens=256)            # Calling the function defined to evaluate the outputs
print(ground2,"\n\n")                                                               # Printing the Groundedness output
print(rel2)                                                                         # Printing the Relevance output

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Output: 5
Explanation: The context clearly states that the treatment for appendicitis involves surgical removal of the appendix due to increased mortality rates with treatment delays. It also mentions that antibiotics are used in conjunction with surgery but not as a cure for appendicitis. Therefore, the answer derived completely from the context. 


 Output: 5
Explanation: The answer directly addresses the question by providing the common symptoms for appendicitis and stating that it cannot be cured via medicine alone, requiring surgical removal instead. The recommended surgical procedures (open or laparoscopic appendectomy) are also mentioned in the answer. Therefore, the answer is fully relevant to the question.


**Observation**: The groundedness rating is 5 and the relevance rating is 5. This means the answer has been derived from the context completely and the response generated covers all aspects of the question even though the model wasn't able to answer 'if appendicitis can be cured via medicine.'

In [72]:
# Output evaluation for Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?
ground3,rel3 = generate_ground_relevance_response(Query3,max_tokens=256)            # Calling the function defined to evaluate the outputs
print(ground3,"\n\n")                                                               # Printing the Groundedness output
print(rel3)                                                                         # Printing the Relevance output

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Output: 5
Explanation: The answer has been derived completely from the context as it mentions the causes of sudden patchy hair loss being alopecia areata and provides the effective treatments for this condition, which were all mentioned in the context. 


 Rating: 5

Explanation: The answer fully addresses the question by providing information on both the causes (alopecia areata) and effective treatments for sudden patchy hair loss mentioned in the question. It covers all aspects of the question, including various treatment options with their potential side effects and the importance of consulting a healthcare professional for an accurate diagnosis and treatment plan.


**Observation**: The groundedness rating is 5 and the relevance rating is 5. This means the RAG model response is completely  derived from the context to generate the response and the response generated is highly relevant to the query as well.

In [73]:
# Output evaluation for Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function??
ground4,rel4 = generate_ground_relevance_response(Query4,max_tokens=256)            # Calling the function defined to evaluate the outputs
print(ground4,"\n\n")                                                               # Printing the Groundedness output
print(rel4)                                                                         # Printing the Relevance output

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Output: 5
Explanation: The context provides a detailed description of the diagnosis, pathology, causes, and treatment options for traumatic brain injuries (TBI). Among the treatments mentioned, supportive care is specifically highlighted as an essential aspect of managing TBI patients. The answer accurately reflects the information provided in the context regarding the importance of supportive care in treating TBIs. 


 Rate 3 - The answer is covering some key aspects of the question and is relevant to the question to a good extent. The answer mentions that there is no specific treatment for traumatic brain injuries but supportive care is important. It also mentions some components of supportive care such as preventing systemic complications, providing good nutrition, and preventing pressure ulcers. However, it does not mention any specific treatments or therapies that may be used during rehabilitation.


**Observation**: The groundedness rating is 5 which means the RAG model response is completely derived from the context to generate the response.

The relevance rating is 3 meaning the Answer is covering only some of the key aspects and answers the questions to a good extent but not completely. Mainly it does not mention any specific treatments or therapies that may be used during rehabilitation.

In [74]:
# Output evaluation for Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?
ground5,rel5 = generate_ground_relevance_response(Query5,max_tokens=256)            # Calling the function defined to evaluate the outputs
print(ground5,"\n\n")                                                               # Printing the Groundedness output
print(rel5)                                                                         # Printing the Relevance output

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Output: 5
Explanation: The answer completely derives from the context as it summarizes and repeats the information provided in the context regarding the necessary precautions and treatment steps for a person who has fractured their leg. 


 Rate 4 - The answer is covering most key aspects of the question and is relevant to the Question mostly. The answer mentions the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, including immobilization using elastic bandages or a splint, elevation of the injured limb, application of warmth after 48 hours, keeping the cast dry, inspecting the edges and skin around the cast, seeking medical care if necessary, and using a splint for stable injuries. However, it does not mention all aspects such as the importance of good hygiene and potential complications of prolonged immobilization or early mobilization.


**Observation**: The groundedness rating is 5 means the RAG model is able to effectively use the context to generate the response and answer is fully grounded.

The relevance rating is 4 meaning the Response is able to answer most aspects of the question but not all. Here the answer does not cover importance of good hygiene and potential complications of prolonged immobilization or early mobilization.

## Actionable Insights and Business Recommendations

**Actionable Insights:**

*   Initial responses from LLM (without Prompt Engineering, Fine Tuning or using RAG model) generated comrehensive responses which seemed accurate. However, the responses were suitable for general audience and sometimes too generic for Healthcare professionals.
*   Using Prompt Engineering made the output responses more concise and incorporated more depth in terms of medical language which a healthcare professional would find more suitable as this model is created for them.
*   RAG model added the context from the Merck Manaul which made the responses even better for the Healthcare professionals as the source of information is better and chances of providing accurate information is even higher.
*   However, if there is any particular query whose answer is not present in the manual, it may cause the RAG model to hallucinate or not provide an answer at all which is not helpful.
*   Fine-tuning the model helps with the responses. Here reducing the randomness or keeping them in check would help e.g., reducing top_k and the top_p value as we want the model to share more facts / accurate information rather than being creative or diverse.


**Business Recomendations:**

*   Instead of only providing one Merck manual, multiple similar trusted sources can be provided for the RAG model to draw context and feed to LLM for better output.
*   Responses from both RAG model and Prompt Engineered output can be shared or summarized in case RAG model is not able to answer any query if the context is not present.
*   Fine-tuning can be done on multiple queries and reviewed with the professionals to understand which combination actually provides the most accurate responses.











<font size=6 color='blue'>Power Ahead</font>
___

In [76]:
!jupyter nbconvert '/content/Full_Code_NLP_RAG_Project_2.ipynb' --to html

[NbConvertApp] Converting notebook /content/Full_Code_NLP_RAG_Project_2.ipynb to html
[NbConvertApp] Writing 584733 bytes to /content/Full_Code_NLP_RAG_Project_2.html
